In [1]:
import spacy

import torch
from torch import nn
import torch.nn.functional as F

from torchtext.datasets import TranslationDataset
from torchtext.data import Field, BucketIterator, TabularDataset
from torch import optim

import time
import random

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:

ARTICLE = Field(tokenize='spacy',
                tokenizer_language='en',
                init_token='<sos>',
                eos_token='<eos>',
                lower=True,
                include_lengths=True
                )
SUMMARY = Field(tokenize='spacy',
                tokenizer_language='en',
                init_token='<sos>',
                eos_token='<eos>',
                lower=True,
                )

In [ ]:
train_data, valid_data, test_data = TabularDataset.splits(
                                                    path="./data",
                                                    train='train.csv',
                                                    validation='val.csv',
                                                    test='test.csv', 
                                                    format='csv',
                                                    fields=[("text",ARTICLE),('headline',SUMMARY)]
                                                        )

In [ ]:
ARTICLE.build_vocab(train_data, min_freq=2)
SUMMARY.build_vocab(train_data, min_freq=2)

In [ ]:
BATCH_SIZE = 64

train_loader, valid_loader, test_loader = BucketIterator.splits(
                                                            (train_data, valid_data, test_data),
                                                            batch_size=BATCH_SIZE,
                                                            sort_within_batch=True,
                                                            sort_key = lambda x:len(x.text),
                                                            device=device
                                                                )